In [1]:
import pandas as pd
import numpy as np
import os
import heapq

In [12]:
# http://www.albertauyeung.com/post/python-matrix-factorization/
class Matrix_Factorizer:
    def __init__(self):
        pass
    def read_matrices(self, size = "medium"):
        # Read_Data: load ratings matrices from csv & numpy data. If data is not available run Pre-Processing first.
        self.matrix_asFrame = pd.read_csv(f"df_ratings_matrix_{size}.csv")
        self.ratings_matrix = np.load("ratings_matrix.npy")
        self.ratings_matrix = np.array(self.ratings_matrix, dtype=np.float64)
        self.num_users, self.num_items = self.ratings_matrix.shape
    def set_factorizer(self, k, alpha, beta, n):
        self.k = k
        self.alpha = alpha
        self.beta = beta
        self.n = n
    def train(self):
        # P is the latent feature matrix for users, Q for items
        self.P = np.random.normal(scale=1./self.k, size=(self.num_users, self.k))
        self.P = np.array(self.P, dtype=np.float64)
        self.Q = np.random.normal(scale=1./self.k, size=(self.num_items, self.k))
        self.Q = np.array(self.Q, dtype=np.float64)
        
        # Init biases for users and items as vectors
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.ratings_matrix[np.where(self.ratings_matrix != 0)])
        
        # Create a list of training samples
        # This model is made for explicit ratings. Therefore we select all items that have a ratings (> 0) for training data.
        # In our implicit model, we dont want that.. 
        """self.samples = [
            (i, j, self.ratings_matrix[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.ratings_matrix[i, j] > 0
        ]"""
        
        # v3.0
        # Add 500 observed and 500 unobserved ratings.
        self.samples = list()
        iterations = 500
        while len(self.samples) < iterations:
            i = np.random.randint(0, self.num_users)
            j = np.random.randint(0, self.num_items)
            observation = self.ratings_matrix[i, j]
            if observation != 0:
                self.samples.append((i, j, observation))
                
        while len(self.samples) < 2*iterations:
            i = np.random.randint(0, self.num_users)
            j = np.random.randint(0, self.num_items)
            observation = self.ratings_matrix[i, j]
            if observation == 0:
                self.samples.append((i, j, observation))
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.n):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mean_squared_error()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process
    def mean_squared_error(self):
        xs, ys = self.ratings_matrix.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.ratings_matrix[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [13]:
Factorizer = Matrix_Factorizer()
Factorizer.read_matrices("small")

In [14]:
Factorizer.ratings_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
# larger parameters have a higher risk of overflow
Factorizer.set_factorizer(30, 0.01, 0.015, 40)
training_process = Factorizer.train()
print()
print("P x Q:")
print(Factorizer.full_matrix())
print()
print("Global bias:")
print(Factorizer.b)
print()
print("User bias:")
print(Factorizer.b_u)
print()
print("Item bias:")
print(Factorizer.b_i)

Iteration: 10 ; error = 8.2603
Iteration: 20 ; error = 13.5945
Iteration: 30 ; error = 17.3064
Iteration: 40 ; error = 19.9953

P x Q:
[[0.98582398 1.00954404 0.99119676 ... 1.00195843 1.00238421 0.99368642]
 [1.0109245  1.00395958 0.9957206  ... 0.99524517 1.00346151 1.00048771]
 [0.58072645 0.58431246 0.57841136 ... 0.59067387 0.57811199 0.58010757]
 ...
 [1.00879687 1.00443415 0.99549035 ... 0.99596487 1.00348752 0.99222891]
 [1.00195785 0.99844991 0.99155556 ... 1.00666698 1.00862283 1.00090781]
 [1.01289548 1.00605657 1.00179615 ... 1.01184435 1.00489713 1.00313534]]

Global bias:
1.0

User bias:
[ 0.00000000e+00  0.00000000e+00 -4.18396272e-01  0.00000000e+00
 -2.75564364e-01 -2.70093668e-01 -2.74070235e-01  0.00000000e+00
  0.00000000e+00 -2.27610901e-03  2.58875019e-02  0.00000000e+00
 -2.85437055e-01  0.00000000e+00 -2.74254000e-01 -2.73207343e-01
 -2.74508815e-01 -2.39333455e-01 -5.47301252e-01 -2.73181471e-01
  0.00000000e+00  0.00000000e+00 -2.74829567e-01 -2.71782439e-01
 

In [36]:
ks = [3, 5, 10, 20, 40, 100, 250]
alphas = [1*10**-1, 1*10**-2, 5*10**-3, 1*10**-4, 5*10**-5]
betas = [15*10**-2, 1*10**-2, 15*10**-3, 15*10**-4]

In [16]:
Factorizer.samples

[(491, 753, 0.0),
 (432, 9509, 1.0),
 (151, 4781, 0.0),
 (370, 7768, 0.0),
 (837, 13302, 1.0),
 (665, 1996, 0.0),
 (734, 12483, 1.0),
 (519, 2593, 1.0),
 (398, 12420, 0.0),
 (10, 263, 1.0),
 (594, 11486, 1.0),
 (153, 4841, 1.0),
 (90, 1377, 1.0),
 (764, 12666, 1.0),
 (250, 5890, 0.0),
 (141, 3238, 1.0),
 (218, 1947, 0.0),
 (238, 10338, 0.0),
 (101, 8359, 0.0),
 (153, 4329, 1.0),
 (129, 7551, 0.0),
 (707, 12172, 1.0),
 (51, 5693, 0.0),
 (397, 11711, 0.0),
 (721, 2847, 0.0),
 (90, 1974, 1.0),
 (332, 13338, 0.0),
 (161, 5272, 1.0),
 (290, 6286, 0.0),
 (525, 11395, 0.0),
 (519, 10643, 1.0),
 (241, 1229, 0.0),
 (847, 13486, 1.0),
 (32, 552, 1.0),
 (689, 8239, 0.0),
 (130, 5566, 0.0),
 (480, 10004, 1.0),
 (288, 2593, 1.0),
 (415, 8693, 1.0),
 (90, 2131, 1.0),
 (725, 4124, 0.0),
 (153, 3775, 1.0),
 (770, 5445, 0.0),
 (230, 6639, 1.0),
 (78, 9593, 0.0),
 (517, 12773, 0.0),
 (288, 7205, 1.0),
 (557, 7697, 0.0),
 (306, 10405, 0.0),
 (94, 2728, 1.0),
 (147, 212, 0.0),
 (334, 8407, 0.0),
 (86, 900